In [1]:
import tensorflow as tf
import os
import keras
import numpy as np
import easygdf
import mne
import json
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, MaxPooling1D, Dropout, Flatten
from tensorflow.keras import layers, models
from tensorflow.keras.regularizers import l2
import get_data
from sklearn.model_selection import train_test_split

First branch

In [ ]:

def squeeze_excitation_block(inputs, r):
    filters = inputs.shape[-1]
    se = layers.GlobalAveragePooling1D()(inputs)
    se = layers.Dense(filters // r, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    se = layers.Reshape((1, filters))(se)
    return layers.multiply([inputs, se])

input_shape = (1000, 22)

inputs = tf.keras.Input(shape=input_shape)

# First Layer
x = layers.Conv1D(32, 20, strides=2, padding='same', kernel_regularizer=l2(0.01))(inputs)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.MaxPooling1D()(x)

# Second Layer
x = layers.Conv1D(32, 9, strides=2, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
layer1_output = layers.MaxPooling1D()(x)

# Third Layer
x = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(layer1_output)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)


# Fourth Layer
x = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.Add()([x, layer1_output])
layer4_output = layers.MaxPooling1D()(x)

# Fifth Layer
x = layers.Conv1D(32, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(layer4_output)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)


# Sixth Layer
x = layers.Conv1D(32, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.Add()([x, layer4_output])
x = layers.MaxPooling1D()(x)

x = layers.Dropout(0.5)(x)

x = layers.Flatten()(x)

outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.summary()



Second branch

In [ ]:
def squeeze_excitation_block(inputs, r):
    filters = inputs.shape[-1]
    se = layers.GlobalAveragePooling1D()(inputs)
    se = layers.Dense(filters // r, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    se = layers.Reshape((1, filters))(se)
    return layers.multiply([inputs, se])

input_shape = (1000, 22)

inputs = tf.keras.Input(shape=input_shape)

# First Layer
x = layers.Conv1D(32, 13, strides=2, padding='same', kernel_regularizer=l2(0.01))(inputs)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)

# Second Layer
x = layers.Conv1D(32, 7, strides=2, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.MaxPooling1D()(x)

# Third Layer
x = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
layer3_output = layers.MaxPooling1D()(x)

# Fourth Layer
x = layers.Conv1D(32, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(layer3_output)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)

# Fifth Layer
x = layers.Conv1D(32, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.Add()([x, layer3_output])
x = layers.MaxPooling1D()(x)

# Dropout layer
x = layers.Dropout(0.5)(x)

# Flatten layer
x = layers.Flatten()(x)

# Output layer
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create model
model = models.Model(inputs, outputs)

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()



Third branch


In [ ]:
def squeeze_excitation_block(inputs, r):
    filters = inputs.shape[-1]
    se = layers.GlobalAveragePooling1D()(inputs)
    se = layers.Dense(filters // r, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    se = layers.Reshape((1, filters))(se)
    return layers.multiply([inputs, se])

input_shape = (1000, 22)

inputs = tf.keras.Input(shape=input_shape)

# First Layer
x = layers.Conv1D(32, 15, strides=2, padding='same', kernel_regularizer=l2(0.01))(inputs)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.MaxPooling1D()(x)

# Second Layer
x = layers.Conv1D(32, 10, strides=2, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)

# Third Layer
x = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.MaxPooling1D()(x)

# Fourth Layer
x = layers.Conv1D(48, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.ELU()(x)
x = squeeze_excitation_block(x, r=8)
x = layers.MaxPooling1D()(x)

# Dropout layer
x = layers.Dropout(0.5)(x)

# Flatten layer
x = layers.Flatten()(x)

# Output layer
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create model
model = models.Model(inputs, outputs)

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

the final version

In [2]:
# Define the squeeze excitation block function
def squeeze_excitation_block(inputs, r):
    filters = inputs.shape[-1]
    se = layers.GlobalAveragePooling1D()(inputs)
    se = layers.Dense(filters // r, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    se = layers.Reshape((1, filters))(se)
    return layers.multiply([inputs, se])

# Set random seed for reproducibility
tf.random.set_seed(42)

# Define input shape
input_shape = (22,1)

# Define input layers for each branch
input_1 = tf.keras.Input(shape=input_shape)
input_2 = tf.keras.Input(shape=input_shape)
input_3 = tf.keras.Input(shape=input_shape)

# First Branch
x1 = layers.Conv1D(32, 20, strides=2, padding='same', kernel_regularizer=l2(0.01))(input_1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ELU()(x1)
x1 = squeeze_excitation_block(x1, r=8)
x1 = layers.MaxPooling1D()(x1)

x1 = layers.Conv1D(32, 9, strides=2, padding='same', kernel_regularizer=l2(0.01))(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ELU()(x1)
x1 = squeeze_excitation_block(x1, r=8)
x1 = layers.MaxPooling1D()(x1)

x1 = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ELU()(x1)

x1 = layers.Flatten()(x1)

# Second Branch
x2 = layers.Conv1D(32, 13, strides=2, padding='same', kernel_regularizer=l2(0.01))(input_2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.ELU()(x2)
x2 = squeeze_excitation_block(x2, r=8)

x2 = layers.Conv1D(32, 7, strides=2, padding='same', kernel_regularizer=l2(0.01))(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.ELU()(x2)
x2 = squeeze_excitation_block(x2, r=8)

x2 = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x2)
x2 = layers.BatchNormalization()(x2)
x2 = layers.ELU()(x2)

x2 = layers.Flatten()(x2)

# Third Branch
x3 = layers.Conv1D(32, 15, strides=2, padding='same', kernel_regularizer=l2(0.01))(input_3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.ELU()(x3)
x3 = squeeze_excitation_block(x3, r=8)

x3 = layers.Conv1D(32, 10, strides=2, padding='same', kernel_regularizer=l2(0.01))(x3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.ELU()(x3)
x3 = squeeze_excitation_block(x3, r=8)

x3 = layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x3)
x3 = layers.BatchNormalization()(x3)
x3 = layers.ELU()(x3)

x3 = layers.Flatten()(x3)

# Concatenate the outputs of three branches
merged = layers.concatenate([x1, x2, x3])

# Dropout layer
x = layers.Dropout(0.5)(merged)

# # Flatten layer
# x = layers.Flatten()(x)

# # Dense layer
# x = layers.Dense(64, activation='relu')(x)

# Output layer
output = layers.Dense(4, activation='softmax')(x)

# Create model
model = Model(inputs=[input_1, input_2, input_3], outputs=output)

# Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 22, 1)]              0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 11, 32)               672       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 11, 32)               128       ['conv1d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 input_2 (InputLayer)        [(None, 22, 1)]              0         []                        

In [3]:
events_with_labels_arrays=get_data.event_position()


Extracting EDF parameters from d:\porojects\EEG classification\datasets\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Extracting EDF parameters from d:\porojects\EEG classification\datasets\BCICIV_2a_gdf\A01E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '783']
Extracting EDF parameters from d:\porojects\EEG classification\datasets\BCICIV_2a_gdf\A02T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EO

C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '783']
Extracting EDF parameters from d:\porojects\EEG classification\datasets\BCICIV_2a_gdf\A03T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Extracting EDF parameters from d:\porojects\EEG classification\datasets\BCICIV_2a_gdf\A03E.gdf...
GDF file dete

...

C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '783']
Extracting EDF parameters from d:\porojects\EEG classification\datasets\BCICIV_2a_gdf\A08T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Extracting EDF parameters from d:\porojects\EEG classification\datasets\BCICI

C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '783']
Extracting EDF parameters from d:\porojects\EEG classification\datasets\BCICIV_2a_gdf\A09T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Extracting EDF parameters from d:\porojects\EEG classification\datasets\BCICI

C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


In [ ]:
data_arrays=get_data.save_data_in_array()

In [5]:

labels = ["A01T", "A02T", "A03T","A04T","A05T","A06T","A07T","A08T","A09T"]

X_train=[]

for label in labels:

    temp_data=data_arrays[label]
    temp_events=events_with_labels_arrays[label]
    for i in range(len(temp_events)):
    
        if temp_events[i,1]==769 or temp_events[i,1]==770 or temp_events[i,1]==771 or temp_events[i,1]==772 :

            ev=temp_events[i,1]
            position=temp_events[i,0]
            for j in range(position, position+1000):

                temp=np.append(temp_data[j,1:23],ev)
                X_train.append(temp)

In [6]:


X_train = np.array(X_train)

#Separate features (X) and labels (y)
X = X_train[:, :-1]  # Extract all rows and all columns except the last one
Y = X_train[:, -1]   # Extract all rows and only the last column

Y = Y.reshape(-1, 1)




In [7]:
for i in range(len(Y)):
    if Y[i]==769:
        Y[i]=0
    elif Y[i]==770:
        Y[i]=1
    elif Y[i]==771:
        Y[i]=2    
    elif Y[i]==772:
        Y[i]=3    

        

In [8]:
x_train, x_, y_train, y_ =train_test_split(X, Y, test_size=0.20, random_state=1)

# x_cv, x_test, y_cv, y_test =train_test_split(x_, y_, test_size=0.50, random_state=1)

# del x_ ,y_

In [9]:
# Generate shuffled indices
indices = np.arange(len(x_train))
np.random.shuffle(indices)

# Shuffle x_train and y_train using the shuffled indices
x_train_shuffled = x_train[indices]
y_train_shuffled = y_train[indices]

# Now x_train_shuffled and y_train_shuffled are shuffled while maintaining correspondence

In [10]:
# Train the model
history=model.fit([x_train_shuffled, x_train_shuffled, x_train_shuffled], y_train_shuffled, epochs=400, batch_size=64, validation_split=0.2)

# Plot the training loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show() 



Epoch 1/400
25920/25920 [==============================] - 228s 8ms/step - loss: 1.4410 - accuracy: 0.2653 - val_loss: 1.3838 - val_accuracy: 0.2754
Epoch 2/400
10376/25920 [===========>..................] - ETA: 2:01 - loss: 1.3839 - accuracy: 0.2746

KeyboardInterrupt: 